# Reading Avro Blobs Into Parquet Data Sets

In [210]:
import os
import string
import json
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession, SQLContext, Window
from pyspark.sql.functions import udf, mean, lit, stddev, col, expr, when, date_sub, avg, window
from pyspark.sql.types import DoubleType, ArrayType, ShortType, LongType, IntegerType, TimestampType, StructType, StringType, StructField
from pyspark.storagelevel import StorageLevel
from azureml.api.schema.dataTypes import DataTypes
from azureml.api.schema.sampleDefinition import SampleDefinition
from azureml.api.realtime.services import generate_schema
from azure.storage.table import TableService, Entity, TablePermissions

STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = os.getenv('TELEMETRY_STORAGE_ACCOUNT_NAME')
TELEMETRY_STORAGE_ACCOUNT_KEY = os.getenv('TELEMETRY_STORAGE_ACCOUNT_KEY')
TABLE_STORAGE_ACCOUNT_NAME = os.getenv('TABLE_STORAGE_ACCOUNT_NAME') #TODO need to add to ENV Variables
TABLE_STORAGE_ACCOUNT_KEY = os.getenv('TABLE_STORAGE_ACCOUNT_KEY') #TODO need to add to ENV Variables
TELEMETRY_CONTAINER_NAME = os.getenv('TELEMETRY_CONTAINER_NAME')
#LOGS_ARCHIVE_CONTAINER_NAME = 'logs-archive' #TODO: introduce environment variables
STAGING_STORAGE_ACCOUNT_NAME = os.getenv('STAGING_STORAGE_ACCOUNT_NAME')
STAGING_STORAGE_ACCOUNT_KEY = os.getenv('STAGING_STORAGE_ACCOUNT_KEY')
LOG_TABLE_NAME = os.getenv('LOG_TABLE_NAME') #TODO need to add this to ENV Variables

In [221]:
#For development purposes only until ENV Variables get set
STORAGE_ACCOUNT_SUFFIX = 'core.windows.net'
TELEMETRY_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
TELEMETRY_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="
TABLE_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
TABLE_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="
TELEMETRY_CONTAINER_NAME = "telemetry"
#LOGS_ARCHIVE_CONTAINER_NAME = "logs" #TODO: introduce environment variables
STAGING_STORAGE_ACCOUNT_NAME = "stguwsjc4vojbrbw"
STAGING_STORAGE_ACCOUNT_KEY = "2GGDWVBtGBy+hAgl5a1uGT4NeU2zzFdocuDFKnOwR2vc5wEOP7jTfbS3/Nl5vvzEudJ4nfH6ozmSOSPXo3xETA=="
LOG_TABLE_NAME = 'cycles'

In [212]:
from pathlib import Path
data_dir = str(Path.home()) + '/data'

#TODO: Convert data_dir into env variable
% rm -rf $data_dir
% mkdir $data_dir $data_dir/logs

In [213]:
wasbTelemetryUrl = "wasb://{0}@{1}.blob.{2}/*/*/*/*/*/*/*".format(TELEMETRY_CONTAINER_NAME, 
                                                                  TELEMETRY_STORAGE_ACCOUNT_NAME, 
                                                                  STORAGE_ACCOUNT_SUFFIX)

sc = SparkSession.builder.getOrCreate()
hc = sc._jsc.hadoopConfiguration()
hc.set("avro.mapred.ignore.inputs.without.extension", "false")
if TELEMETRY_STORAGE_ACCOUNT_KEY:
     hc.set("fs.azure.account.key.{}.blob.core.windows.net".format(TELEMETRY_STORAGE_ACCOUNT_NAME), TELEMETRY_STORAGE_ACCOUNT_KEY)
hc.set("fs.azure.account.key.{}.blob.core.windows.net"
    .format(STAGING_STORAGE_ACCOUNT_NAME), STAGING_STORAGE_ACCOUNT_KEY)
sql = SQLContext.getOrCreate(sc)
avroblob = sql.read.format("com.databricks.spark.avro").load(wasbTelemetryUrl)
avroblob.show()

+--------------------+----------+--------------------+--------------------+
|     EnqueuedTimeUtc|Properties|    SystemProperties|                Body|
+--------------------+----------+--------------------+--------------------+
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T16:41:...|     Map()|Map(connectionAut...|[7B 22 6D 61 63 6...|
|2018-06-20T

In [214]:
#Convert byteformat to string format in pyspark dataframe
from json import loads as Loads
column = avroblob['Body']
string_udf = udf(lambda x: x.decode("utf-8"))
avroblob=avroblob.withColumn("BodyString", string_udf(column))
avroblob.printSchema()

root
 |-- EnqueuedTimeUtc: string (nullable = true)
 |-- Properties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- SystemProperties: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- Body: binary (nullable = true)
 |-- BodyString: string (nullable = true)



In [215]:
#Convert dataframe into 
new_df = sql.read.json(avroblob.select("BodyString").rdd.map(lambda r: r.BodyString))
new_df.show()

+----------------+-------------------+-----------+--------+-------+-------------+-----------+--------------------+---------+
|ambient_pressure|ambient_temperature|  machineID|pressure|  speed|speed_desired|temperature|           timestamp|vibration|
+----------------+-------------------+-----------+--------+-------+-------------+-----------+--------------------+---------+
|          100.92|              20.06|Machine-004|  655.58|1133.27|         1000|     174.62|2018-06-20T16:41:...|     null|
|          101.02|              19.95|Machine-001|   46.46|  -2.17|            0|     164.84|2018-06-20T16:41:...|     null|
|          101.04|              20.06|Machine-004|  673.66|1131.69|         1000|     174.71|2018-06-20T16:41:...|     null|
|          101.07|              20.05|Machine-001|   70.93|   1.97|            0|     164.95|2018-06-20T16:41:...|     null|
|           101.1|               19.9|Machine-004|  660.65|1138.23|         1000|     174.63|2018-06-20T16:41:...|     null|


In [216]:
new_df.write.parquet(data_dir+"/telemetry", mode="overwrite")

In [217]:
#table retrieval
table_service = TableService(account_name=TABLE_STORAGE_ACCOUNT_NAME, account_key=TABLE_STORAGE_ACCOUNT_KEY)

In [222]:
#get logs?
tblob = table_service.query_entities(LOG_TABLE_NAME)

In [223]:
attributes = list()
for row in tblob:
    if (len(attributes) == 0):
        for attribute in row:
            attributes.append(attribute)
    break
log_df = pd.DataFrame(columns=attributes)
for row in tblob:
    row_dict = {}    
    for attribute in row:
        if (attribute != "Timestamp"):
            row_dict[attribute] = row[attribute]
        else:
            newtime = row[attribute].replace(tzinfo=None)
            timeitem = pd.Timestamp(newtime, tz=None)
            row_dict[attribute] = timeitem
    log_df = log_df.append(row_dict, ignore_index=True)
del log_df["RollingWindow"]
log_df.head()

,SpeedAvg,PartitionKey,etag,TemperatureMax,RowKey,SpeedDesiredMax,RawCount,CycleEnd,PressureAvg,Timestamp,PressureMax,TemperatureAvg
0,1112.203945,Machine-000,"W/""datetime'2018-06-19T18%3A09%3A07.5890441Z'""",137.88,2018-06-19 18:02:15.669,1000.0,578,2018-06-19 18:07:05.214,1484.384498,2018-06-19 18:09:07.589044,1545.88,134.648235
1,956.581250,Machine-000,"W/""datetime'2018-06-19T18%3A11%3A41.9128297Z'""",137.91,2018-06-19 18:08:06.23,1000.0,240,2018-06-19 18:10:06.761,1302.244250,2018-06-19 18:11:41.912829,1544.61,137.803750
2,765.731667,Machine-000,"W/""datetime'2018-06-19T18%3A13%3A50.288937Z'""",137.92,2018-06-19 18:11:07.768,1000.0,120,2018-06-19 18:12:07.022,1071.589333,2018-06-19 18:13:50.288937,1544.25,137.835000
3,1020.737889,Machine-000,"W/""datetime'2018-06-19T18%3A17%3A52.8713843Z'""",137.96,2018-06-19 18:13:08.03,1000.0,360,2018-06-19 18:16:09.293,1374.879333,2018-06-19 18:17:52.871384,1543.93,137.856500
4,1021.222056,Machine-000,"W/""datetime'2018-06-19T18%3A21%3A58.1727587Z'""",138.00,2018-06-19 18:17:10.299,1000.0,360,2018-06-19 18:20:10.33,1372.087889,2018-06-19 18:21:58.172758,1543.10,137.885667
